# Three-step RAS expansion for the $1s^2 2s^2\: ^1S_0$ and $1s^2 2s2p\: ^{1,3}P_1$ levels of neutral beryllium

In [2]:
using JAC


In JAC, an restricted-active space (RAS) expansion is to a particular (atomic) representation and, more generally, refers to systematically enlarged SCF computations that are performed stepwise in order to include additional layers of active orbitals. RAS computations are based on one several *reference configurations*, a well-defined *level symmetry* for all CSF bases under consideration as well as on virtual single- (S), double- (D), triple-excitations (T), etc. of electrons from valence orbitals $\to$ valence+virtual orbitals from the (so-called) *active* set. These SD++ excitations can be chosen differently for each step of the RAS computation, similar as the orbitals which are kept *frozen* during a particular step. More often than not, another layer of active orbitals is added and optimized in each step, while all orbitals from previous steps are kept frozen.

In this tutorial, we explain how a three-step RAS expansion for the $1s^2 2s^2\: ^1S_0$ ground level of neutral beryllium as well as for its two lowest-excited $1s^2 2s2p\: ^{1,3}P_1$ levels can be readily generated. For such a representation (of one or several atomic states), we first compute the orbitals for the $1s^2 2s^2$ reference configuration (in some configuration-average approach), and then stepwise enlarge the active orbitals by $2s,\: 2p$ (step$_1$), $3s,\: 3p,\: 3d$ (step$_2$), and finally $4s,\: ...,\: 4f$ (step$_3$). In all these steps, the $1s$ orbitals as well as the orbitals from the previous steps are kept frozen to ensure a proper convergence of the orbitals.

Before we shall explain further details about such RAS expansions (and computations), however, let us however look again at the `struct Atomic.Representation` from which the RAS representation can be *generate*(d) as usual:

In [5]:
? Atomic.Representation

`struct  Atomic.Representation`       ... a struct for defining an atomic representation. Such representations often refer to approximate wave function approximations of         one or several levels but may concern also a mean-field basis (for some multiplet of some given configurations) or Green functions,         etc.

```
+ name             ::String                      ... to assign a name to the given model.
+ nuclearModel     ::Nuclear.Model               ... Model, charge and parameters of the nucleus.
+ grid             ::Radial.Grid                 ... The radial grid to be used for the computation.
+ refConfigs       ::Array{Configuration,1}      ... List of references configurations, at least 1.
+ repType          ::AbstractRepresentationType  ... Specifies the particular representation.
```

---

`Atomic.Representation()`  ... constructor for an 'empty' instance of the a variable::Atomic.Representation


For a RAS expansion, the `repType` must be a `RasExpansion()`, while all virtual excitations of the stepwise enlarged computations refer to the (list of) reference configurations `refConfigs`.

In [6]:
? Atomic.RasExpansion

`struct  Atomic.RasExpansion    <:  AbstractRepresentationType`       ... a struct to represent (and generate) a restricted active-space representation.

```
+ symmetry         ::LevelSymmetry             ... Symmetry of the levels/CSF in the many-electron basis.
+ NoElectrons      ::Int64                     ... Number of electrons.
+ steps            ::Array{Atomic.RasStep,1}   ... List of SCF steps that are to be done in this model computation.
+ settings         ::Atomic.RasSettings        ... Settings for the given RAS computation
```

---

`Atomic.RasExpansion()`  ... constructor for an 'empty' instance of the a variable::Atomic.RasExpansion


In this RAS expansion, we have to specify the (level) `symmetry` of automtically generated CSF expansions as well as *list of* `steps`, which specify the virtual excitations as well as those orbitals which are kept to be frozen during this particular step. These steps are *decribed* by:

In [7]:
? RasStep

search: RasStep RasSettings uppercasefirst lowercasefirst



`struct  Atomic.RasStep`       ... specifies an individual step of a (relativistic) restricted active space computation for a set of levels. This struct          comprises all information to generate the orbital basis and to perform the associated SCF and multiplet computations for a          selected number of levels.

```
+ seFrom            ::Array{Shell,1}        ... Single-excitations from shells   [sh_1, sh_2, ...]
+ seTo              ::Array{Shell,1}        ... Single-excitations to shells  [sh_1, sh_2, ...]
+ deFrom            ::Array{Shell,1}        ... Double-excitations from shells   [sh_1, sh_2, ...]
+ deTo              ::Array{Shell,1}        ... Double-excitations to shells  [sh_1, sh_2, ...]
+ teFrom            ::Array{Shell,1}        ... Triple-excitations from shells   [sh_1, sh_2, ...]
+ teTo              ::Array{Shell,1}        ... Triple-excitations to shells  [sh_1, sh_2, ...]
+ qeFrom            ::Array{Shell,1}        ... Quadrupole-excitations from shells   [sh_1, sh_2, ...]
+ qeTo              ::Array{Shell,1}        ... Quadrupole-excitations to shells  [sh_1, sh_2, ...]
+ frozenShells      ::Array{Shell,1}        ... List of shells that are kept 'frozen' in this step.
+ constraints       ::Array{String,1}       ... List of Strings to define 'constraints/restrictions' to the generated CSF basis.
```

---

`Atomic.RasStep()`  ... constructor for an 'empty' instance of a variable::Atomic.RasStep

---

`Atomic.RasStep(rasStep::Atomic.RasStep;`

```
                seFrom::Array{Shell,1}=Shell[], seTo::Array{Shell,1}=Shell[], 
                deFrom::Array{Shell,1}=Shell[], deTo::Array{Shell,1}=Shell[], 
                teFrom::Array{Shell,1}=Shell[], teTo::Array{Shell,1}=Shell[], 
                qeFrom::Array{Shell,1}=Shell[], qeTo::Array{Shell,1}=Shell[], 
                frozen::Array{Shell,1}=Shell[], constraints::Array{String,1}=String[]  
                
... constructor for modifying the given rasStep by specifying all excitations, frozen shells and constraints optionally.
```


with quite an obvious meaning of single excitations (se), double excitations (de), etc., and where `from` and `to` refer lists of active shells. For each step, we can also specify which `frozenShells` are kept frozen and which additional constraints should be taken into account in generating the CSF lists. **We note, however, that presently no constraints are taken into account in these RAS expansions.** 

Apart from the `steps`, various details about the RAS computations can be specified in terms of the `settings::RasSettings` above, and which apply (equally) to *all* steps:

In [8]:
? RasSettings

search: RasSettings



`struct  Atomic.RasSettings`       ... a struct for defining the settings for a restricted active-space computations.

```
+ levelsScf            ::Array{Int64,1}     ... Levels on which the optimization need to be carried out.
+ maxIterationsScf     ::Int64              ... maximum number of SCF iterations in each RAS step.
+ accuracyScf          ::Float64            ... convergence criterion for the SCF field.

+ breitCI              ::Bool               ... logical flag to include Breit interactions.
+ selectLevelsCI       ::Bool               ... true, if specific level (number)s have been selected.
+ selectedLevelsCI     ::Array{Int64,1}     ... Level number that have been selected.
```

---

`Atomic.RasSettings()`  ... constructor for setting the default values.


These settings enables one to specify the levels on which the SCF is optimized as well as the maximum number of iterations and requested accuracy in these computations. In addition, the flag `breitCI` determines the use of the Breit interaction in the set-up (and diagonalization) of the Hamiltonian matrix; this flag does not affect the SCF of the *next* RAS step but may influence further computations, if the multiplets of the individual steps are subsequently used in computations of properties and processes. We can look for the default settings as well as specify these settings explictly by:

In [9]:
Atomic.RasSettings()

levelsScf:            [1]  
maxIterationsScf:     24  
accuracyScf:          1.0e-6  
breitCI:              false  
selectLevelsCI:       false  
selectedLevelsCI:     Int64[]  


In [10]:
rasSettings = RasSettings([1], 24, 1.0e-6, false, true, [1,2,3] )

levelsScf:            [1]  
maxIterationsScf:     24  
accuracyScf:          1.0e-6  
breitCI:              false  
selectLevelsCI:       true  
selectedLevelsCI:     [1, 2, 3]  


The last two entries to these settings tell JAC that we wish to compute (and print) always the lowest three levels of each Hamiltonian matrix; **this feature is done properly at present since, until now, the whole Hamiltonian matrix is diagonalized (and printed) in all cases.**

We are now prepared to define the RAS computation as a whole, i.e. a name (mainly for the printout), the reference configuration(s) as well as in turn all (three) steps of a RAS computation for the $1s^2 2s^2\: ^1S_0$ ground level of neutral beryllium:

In [ ]:
name        = "Beryllium 1s^2 2s^2 ^1S_0 ground state"
refConfigs  = [Configuration("[He] 2s^2")]

We also specify that we will keep the $1s$ orbital frozen in the computation of all orbitals beyond those in the reference configurations as well as that all excitations are to be taken from the $2s$ shell:

In [ ]:
frozen      = [Shell("1s")]
from        = [Shell("2s")]

In [ ]:
to          = [Shell("2s"), Shell("2p")]
step1       = RasStep(RasStep(); seFrom=from, seTo=deepcopy(to), deFrom=from, deTo=deepcopy(to), frozen=deepcopy(frozen))

Similarly, we can specify the second and third step by using (and overwriting) the information provided for the previous step; the use of deepcopy() ensures that no internal calls are repeated if these cells are called in different order:

In [ ]:
append!(frozen, [Shell("2s"), Shell("2p")])
append!(to,     [Shell("3s"), Shell("3p"), Shell("3d")])
step2       = RasStep(step1; seTo=deepcopy(to), deTo=deepcopy(to), frozen=deepcopy(frozen))

In [ ]:
append!(frozen, [Shell("3s"), Shell("3p"), Shell("3d")])
append!(to,     [Shell("4s"), Shell("4p"), Shell("4d"), Shell("4f")])
step3       = RasStep(step2; seTo=deepcopy(to), deTo=deepcopy(to), frozen=deepcopy(frozen))

With these definition of all *steps*, we can now define the RAS computations as a whole and subsequently *run* it:

In [ ]:
wa          = RasComputation(name, Nuclear.Model(4.), Radial.Grid("grid: exponential"), refConfigs, LevelSymmetry(0, Basics.plus),
                             4, [step1, step2, step3], rasSettings)

In [ ]:
wb = perform(wa, output=true)

Of course, analogue computations can be readily carried out also for the (two) $1s^2 2s2p\: ^{1,3}P_1$ excited levels of neutral beryllium, and where we use major parts of the previous computation:

In [ ]:
name2        = "Beryllium 1s^2 2s^2 ^1S_0 ground state"
refConfigs2  = [Configuration("[He] 2s^2")]
wa2          = RasComputation(name2, Nuclear.Model(4.), Radial.Grid("grid: exponential"), refConfigs2, LevelSymmetry(1, Basics.minus),
                              4, [step1, step2, step3], rasSettings)

In [ ]:
wb2 = perform(wa2, output=true)

All *results* of this and the previous RAS computations are kept by the dictionaries `wb2` and `wb` above. We can use these variables to extract, for instance, the multiplets as obtained from the first step:

In [ ]:
wm = wb["reference multiplet"];   wm2 = wb2["reference multiplet"]

In [ ]:
Obviously, the describe the same *level* since they are both built just open the $1s^2 2s^2$ closed-shell configuration:

In [ ]:
println("Energies from references conf:  $(wm.levels[1].energy)   $(wm2.levels[1].energy)   ")

These energies differ, however, if we compare them for different steps as well as for the two different RAS computations.

In [ ]:
wm = wb["step1"];   wm2 = wb2["step1"]
println("Energies from step 1:           $(wm.levels[1].energy)   $(wm2.levels[1].energy)   ")

In [ ]:
wm = wb["step2"];   wm2 = wb2["step2"]
println("Energies from step 2:           $(wm.levels[1].energy)   $(wm2.levels[1].energy)   ")

In [ ]:
wm = wb["step3"];   wm2 = wb2["step3"]
println("Energies from step 3:           $(wm.levels[1].energy)   $(wm2.levels[1].energy)   ")